In [ ]:
# Necessary downloads to run the code
!pip install transformers torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import all packages
import transformers
from transformers import AutoTokenizer, PhiForCausalLM
import re
from tqdm import tqdm
import time
import os

/home/jhensman/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load in the model and tokenizer for Fietje-2 model
model_name = "BramVanroy/fietje-2"
model = PhiForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.06it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Function for readin the file
def read_file(filename):
    with open(filename, 'r') as file:
        content = file.read()
    return content

# Function for splitting the texts into sections and paragraphs
def split_text_into_sections(text):
    # Split the text based on the two white lines
    parts = re.split(r'\n\n\n', text)
    sections = []

    # For each section with headers split it into multiple paragraphs
    for part in parts:
        current_section = {}
        paragraphs = part.split('\n\n')
        for x in range(len(paragraphs)):
          lines = paragraphs[x].split('\n')
          if x == 0:
            current_section['heading'] = lines[0]
            current_section['paragraphs'] = [paragraphs[x][len(lines[0])+1:]]
          else:
            current_section['paragraphs'].append(paragraphs[x])
        sections.append(current_section)
    return sections

# Function for simplifying text with the appraoch few-shot prompting
def simplify_paragraph(text, max_length=500):
    if not text.strip():
        return ""

    prompt = f"""
        Ik wil dat je mijn complexe zin vervangt met een simpele zin.
        De betekenis van de zin moet hetzelfde blijven, maar maak het makkelijker.
        Complex: Ten aanzien van de naleving van de nieuwe wetgeving is het van belang dat alle betrokken partijen op de hoogte zijn van de specifieke verplichtingen en verantwoordelijkheden die hieruit voortvloeien.
        Simpel: Voor de naleving van de nieuwe wet is het belangrijk dat iedereen weet welke verplichtingen en verantwoordelijkheden er zijn.
        Complex: De overheid heeft besloten dat, in het kader van de nieuwe subsidieregeling, organisaties die in aanmerking willen komen voor financiële ondersteuning, moeten voldoen aan een reeks strengere criteria, welke zijn opgesteld om de efficiëntie en effectiviteit van de verstrekte subsidies te waarborgen.
        Simpel: De overheid heeft nieuwe regels voor subsidies. Organisaties die subsidies willen, moeten nu aan strengere criteria voldoen. Deze criteria zorgen ervoor dat subsidies beter worden gebruikt.
        Complex: {text}
        Simpel:
        """
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    text_tokenized = tokenizer.encode(text, return_tensors="pt")

    # Define the maximum length
    input_length = len(inputs[0])
    text_length = len(text_tokenized[0])
    max_length = int(input_length + text_length)

    # Generate the output if possible, otherwise return original paragraph
    try:
        outputs = model.generate(
            inputs,
            max_length=max_length,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
        )
    except ValueError as e:
        print(f"Error processing paragraph: {text}")
        print(f"Error: {e}")
        return text

    # Decode the output
    simplified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the simplified text from the generated text
    simplified_text = simplified_text.replace(prompt, "").strip()

    if "Complex:" in simplified_text:
        simplified_text = simplified_text.split("Complex:")[0].strip()

    return simplified_text

# Build the new text
def rebuild_text(sections):
    new_text = ''
    total_sections = len(sections)
    process_text = True
    # Start the timer
    start_time = time.time()

    # Start the progress bar
    with tqdm(total=total_sections, desc="Processing Sections") as pbar:
        for section in sections:
            # Copy the header
            heading = section['heading']
            new_text += heading + '\n\n'

            # If it is the last header, do not simplify further
            if heading == "Met vriendelijke groet,":
                process_text = False

            # simplify the paragraphs and paste both the original and simplified paragraphs
            for paragraph in section['paragraphs']:
                if process_text:
                    new_text +=  'Original: \n' + paragraph + '\n\n'
                    new_text += 'Simplified: \n' + simplify_paragraph(paragraph) + '\n\n'
                else:
                    new_text +=  paragraph + '\n'
            # Update the progress bar
            pbar.update(1)

    # Stop the timer
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Processing completed in {elapsed_time:.2f} seconds.")

    return new_text.strip()

# Function for simplifying a letter and writing it to a file
def simplify_letter(text, output_file):
    sections = split_text_into_sections(text)
    new_text = rebuild_text(sections)
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(new_text)

# Function for generating the filename for the output
def generate_filename(input_filename):
    filename_without_extension = input_filename.rsplit('.', 1)[0]
    output_filename = "simplified/" + filename_without_extension + '_raw_simplified.txt'
    return output_filename

# Function to loop through all files in the folder
def process_all_files_in_folder(folder_path):
    files = os.listdir(folder_path)

    for file in tqdm(files, desc="Processing files"):
        full_file_path = os.path.join(folder_path, file)
        # Ensure it's a file
        if os.path.isfile(full_file_path):
            text = read_file(full_file_path)
            output_file = generate_filename(file)
            simplify_letter(text, output_file)

In [ ]:
# Process all files in the specified folder
path = 'data_formatted_part9/'
process_all_files_in_folder(path)

Processing files:  40%|████      | 2/5 [02:41<04:01, 80.51s/it]

Processing completed in 161.03 seconds.



Processing files:  60%|██████    | 3/5 [08:02<06:02, 181.12s/it]

Processing completed in 321.96 seconds.



Processing files:  80%|████████  | 4/5 [10:21<02:45, 165.14s/it]

Processing completed in 138.31 seconds.



Processing files: 100%|██████████| 5/5 [15:59<00:00, 191.85s/it]

Processing completed in 337.96 seconds.
